# Extracting Features from WAV Files

In [29]:
# Load required libraries
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages')
import librosa
import librosa.display
import soundfile
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np

In [78]:
# Load the sample WAV file
audio_file = 'sample_audio/debussy.wav'
audio, sample_rate = librosa.load(audio_file)

In [79]:
# Set the default frame size and hop length
FRAME_SIZE = 2048
HOP_SIZE = 512

## Time-Domain Features

### Estimated Tempo

In [161]:
bpm_audio = librosa.beat.tempo(y=audio, sr=sample_rate, hop_length=HOP_SIZE)

### Amplitude Envelope

In [80]:
def amplitude_envelope(signal, frame_size, hop_length):
    return np.array([max(signal[i:(i + frame_size)]) for i in range(0, len(signal), hop_length)])

In [81]:
ae_audio = amplitude_envelope(audio, FRAME_SIZE, HOP_SIZE)
ae_mean = np.mean(ae_audio)
ae_stddev = np.std(ae_audio)

In [205]:
ae_audio.shape

(1292,)

### Root-Mean Square Energy

In [82]:
rms_audio = librosa.feature.rms(audio, frame_length=FRAME_SIZE, hop_length=HOP_SIZE)[0]
rms_mean = np.mean(rms_audio)
rms_stddev = np.std(rms_audio)

In [204]:
rms_audio.shape

(1292,)

### Energy Entropy

In [166]:
def compute_energy(frame):
    return np.sum(np.abs(frame)**2) / len(frame)


def compute_frame_energy_entropy(frame, num_subframes=20):
    subframe_size = int(np.floor(len(frame) / num_subframes))
    subframes = [frame[i:(i + subframe_size)] for i in range(0, len(frame), subframe_size)]
    
    energy = np.array([compute_energy(subframe) for subframe in subframes])
    energy = energy / np.sum(energy)
    
    return -np.sum(energy * np.log2(energy))
    

def energy_entropy(signal, frame_length, hop_length):
    return np.array([compute_frame_energy_entropy(signal[i:(i + frame_length)]) for i in range(0, len(signal), hop_length)])

In [172]:
ee_audio = energy_entropy(audio, FRAME_SIZE, HOP_SIZE)
ee_mean = np.mean(ee_audio)
ee_stddev = np.std(ee_audio)

In [203]:
ee_audio.shape

(1292,)

### Zero-Crossing Rate

In [83]:
zcr_audio = librosa.feature.zero_crossing_rate(audio, frame_length=FRAME_SIZE, hop_length=HOP_SIZE)[0]
zcr_mean = np.mean(zcr_audio)
zcr_stddev = np.std(zcr_audio)

In [202]:
zcr_audio.shape

(1292,)

## Frequency-Domain Features

In [84]:
# Extract the spectrograms
audio_spctgm = librosa.stft(audio, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)

### Band-Energy Ratio

In [85]:
SPLIT_FREQ = 2000

In [175]:
def calculate_split_frequency_bin(spectrogram, split_frequency, sample_rate):
    frequency_range = sample_rate / 2
    frequency_delta_per_bin = frequency_range / spectrogram.shape[0]
    split_frequency_bin = np.floor(split_frequency / frequency_delta_per_bin)
    return int(split_frequency_bin)


def to_power_spectrogram(spectrogram):
    return (np.abs(spectrogram) ** 2)


def calculate_ber_for_frame(frequencies_in_frame, split_frequency_bin):
    sum_power_low_freq = np.sum(frequencies_in_frame[:split_frequency_bin])
    sum_power_high_freq = np.sum(frequencies_in_frame[split_frequency_bin:])
    return (sum_power_low_freq / sum_power_high_freq)

    
def band_energy_ratio(spectrogram, split_frequency, sample_rate):
    split_frequency_bin = calculate_split_frequency_bin(spectrogram, split_frequency, sample_rate)
    power_spec = to_power_spectrogram(spectrogram).T
    ber = [calculate_ber_for_frame(freqs_in_frame, split_frequency_bin) for freqs_in_frame in power_spec]
    return np.array(ber)

In [176]:
ber_audio = band_energy_ratio(audio_spctgm, SPLIT_FREQ, sample_rate)
ber_mean = np.mean(ber_audio)
ber_stddev = np.std(ber_audio)

In [201]:
ber_audio.shape

(1292,)

### Spectral Centroid

In [96]:
sc_audio = librosa.feature.spectral_centroid(audio, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
sc_mean = np.mean(sc_audio)
sc_stddev = np.std(sc_audio)

In [200]:
sc_audio.shape

(1292,)

### Bandwidth / Spectral Spread

In [99]:
ss_audio = librosa.feature.spectral_bandwidth(audio, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
ss_mean = np.mean(ss_audio)
ss_stddev = np.std(ss_audio)

In [199]:
ss_audio.shape

(1292,)

### Spectral Rolloff

In [101]:
sroll_audio = librosa.feature.spectral_rolloff(audio, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
sroll_mean = np.mean(sroll_audio)
sroll_stddev = np.std(sroll_audio)

In [198]:
sroll_audio.shape

(1292,)

### Spectral Flatness

In [155]:
sflat_audio = librosa.feature.spectral_flatness(audio, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
sflat_mean = np.mean(sflat_audio)
sflat_stddev = np.std(sflat_audio)

In [197]:
sflat_audio.shape

(1292,)

### Spectral Contrast

In [158]:
sconstrast_audio = librosa.feature.spectral_contrast(audio, sr=sample_rate, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
sconstrast_mean = np.mean(sconstrast_audio)
sconstrast_stddev = np.std(sconstrast_audio)

In [196]:
sconstrast_audio.shape

(1292,)

### Spectral Flux

https://www.sciencedirect.com/topics/engineering/spectral-flux

In [152]:
def normalize(spectra):
    return spectra / np.sum(spectra)

def compute_spect_flux_for_frame(prev_spectra_nmlzd, curr_spectra):
    curr_spectra_nmlzd = normalize(curr_spectra)
    frame_sf = np.linalg.norm(curr_spectra_nmlzd - prev_spectra_nmlzd)
    return frame_sf, curr_spectra_nmlzd

def spectral_flux(spectrogram):
    num_frames = spectrogram.shape[1]
    
    curr_spectra = spectrogram[:, 0]
    curr_spectra_nmlzd = normalize(curr_spectra)

    sf = []
    for i in range(1, num_frames):
        frame_sf, curr_spectra_nmlzd = compute_spect_flux_for_frame(curr_spectra_nmlzd, spectrogram[:, i])
        sf.append(frame_sf)
        
    return np.array(sf)

In [151]:
sf_audio = spectral_flux(audio_spctgm)
sf_mean = np.mean(sf_audio)
sf_stddev = np.std(sf_audio)

In [195]:
sf_audio.shape

(1291,)

## Mel-Frequency Cepstral Coefficients

In [111]:
mfccs_audio = librosa.feature.mfcc(audio, n_mfcc=13, sr=sample_rate)
mfccs_mean = np.mean(mfccs_audio, axis=1)
mfccs_stddev = np.std(mfccs_audio, axis=1)

In [194]:
mfccs_audio.shape

(13, 1292)

## Chroma Vector

In [119]:
chroma_audio = librosa.feature.chroma_stft(y=audio, sr=sample_rate, n_chroma=12, 
                                           n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
chroma_mean = np.mean(chroma_audio, axis=1)
chroma_stddev = np.std(chroma_audio, axis=1)

In [206]:
chroma_mean

array([0.3427883 , 0.14246514, 0.3568891 , 0.18810047, 0.38711867,
       0.20369092, 0.17791696, 0.53722024, 0.2640053 , 0.19303429,
       0.13443361, 0.26362732], dtype=float32)

## Complete Audio Extraction

In [265]:
import librosa
import numpy as np

class AudioFeatureExtractor():
    MEAN = '_mean'
    STD = '_std'

    TEMPO = 'tempo'
    TIME_DOMAIN = ['amplitude_envelope', 'energy_entropy', 'zero_crossing_rate']
    FREQ_DOMAIN = ['band_energy_ratio', 'spectral_centroid', 'spectral_bandwidth',
                   'spectral_rolloff', 'spectral_flatness', 'spectral_contrast',
                   'spectral_flux']
    MFCC = 'mfcc'
    N_MFCC = 13
    CHROMA = 'chroma'
    N_CHROMA = 12
    GENRE = 'genre'

    def __init__(self, frame_size=1024, hop_length=512):
        self.frame_size=frame_size
        self.hop_length=hop_length
#         self._generate_input_feature_list()

#     def _generate_input_feature_list(self):
#         self.input_features = [AudioFeatureExtractor.TEMPO]
#         for time_feature in AudioFeatureExtractor.TIME_DOMAIN:
#             self._append_ml_features(time_feature)
#         for freq_feature in AudioFeatureExtractor.FREQ_DOMAIN:
#             self._append_ml_features(freq_feature)
#         self._append_vector_features(AudioFeatureExtractor.N_MFCC, AudioFeatureExtractor.MFCC)
#         self._append_vector_features(AudioFeatureExtractor.N_CHROMA, AudioFeatureExtractor.CHROMA)
#         self.input_features.append(AudioFeatureExtractor.GENRE)

#     def _append_vector_features(self, n, base_feature):
#         for i in range(1, n + 1):
#             audio_feature = base_feature + str(i)
#             self.input_features.append(audio_feature + AudioFeatureExtractor.MEAN)

#         for i in range(1, n + 1):
#             audio_feature = base_feature + str(i)
#             self.input_features.append(audio_feature + AudioFeatureExtractor.MEAN)


#     def _append_ml_features(self, audio_feature):
#         self.input_features.append(audio_feature + AudioFeatureExtractor.MEAN)
#         self.input_features.append(audio_feature + AudioFeatureExtractor.STD)


### Feature Extraction

    def extract(self, filename, genre):
        signal, sr = librosa.load(filename)

        tempo = self._tempo(signal, sr)
        ae, rms, ee, zcr = self._extract_time_features(signal)
        ber, s_centr, sb, s_roll, s_flat, s_contr, s_flux = self._extract_frequency_features(signal, sr)
        mfcc = self._mfcc(signal, sr)
        chroma = self._chroma(signal, sr)

        feature_matrix = np.vstack([ae, rms, ee, zcr, ber, s_centr, sb, s_roll, s_flat, s_contr, mfcc, chroma])

        return self._compile_data_vector(tempo, feature_matrix, s_flux, genre)

    def _extract_time_features(self, signal):
        ae = self._amplitude_envelope(signal)
        rms = self._root_mean_square_energy(signal)
        ee = self._energy_entropy(signal)
        zcr = self._zero_crossing_rate(signal)
        return ae, rms, ee, zcr

    def _extract_frequency_features(self, signal, sr):
        spectrogram = self._extract_spectrogram(signal)

        ber = self._band_energy_ratio(spectrogram, sr)
        s_centr = self._spectral_centroid(signal, sr)
        sb = self._spectral_bandwidth(signal, sr)
        s_roll = self._spectral_rolloff(signal, sr)
        s_flat = self._spectral_flatness(signal)
        s_contr = self._spectral_contrast(signal, sr)
        s_flux = self._spectral_flux(spectrogram)

        return ber, s_centr, sb, s_roll, s_flat, s_contr, s_flux

    def _compile_data_vector(self, tempo, feature_matrix, s_flux, genre):
        means = np.mean(feature_matrix, axis=1)
        std_devs = np.std(feature_matrix, axis=1)
        return np.hstack([tempo, means, np.mean(s_flux), std_devs, np.std(s_flux), genre])

### Time-Domain Features

    # Tempo (beats per minute)
    def _tempo(self, signal, sr):
        return librosa.beat.tempo(y=signal, sr=sr, hop_length=self.hop_length)

    # Amplitude Envelope
    def _amplitude_envelope(self, signal):
        return np.array([max(signal[i:(i + self.frame_size)]) for i in range(0, len(signal), self.hop_length)])

    # Root-Mean Square Energy
    def _root_mean_square_energy(self, signal):
        return librosa.feature.rms(signal, frame_length=self.frame_size, hop_length=self.hop_length)[0]
    
    # Energy Entropy
    def _energy_entropy(self, signal):
        return np.array([self._compute_frame_energy_entropy(signal[i:(i + self.frame_size)]) for i in range(0, len(signal), self.hop_length)])

    def _compute_frame_energy_entropy(self, frame, num_subframes=20):
        subframe_size = int(np.floor(len(frame) / num_subframes))
        subframes = [frame[i:(i + subframe_size)] for i in range(0, len(frame), subframe_size)]
        
        energy = np.array([self._compute_energy(subframe) for subframe in subframes])
        energy = energy / np.sum(energy)
        
        return -np.sum(energy * np.log2(energy))

    def _compute_energy(self, frame):
        return np.sum(np.abs(frame)**2) / len(frame)

    # Zero Crossing Rate
    def _zero_crossing_rate(self, signal):
        return librosa.feature.zero_crossing_rate(signal, frame_length=self.frame_size, hop_length=self.hop_length)[0]

    
### Frequency-Domain Features

    def _extract_spectrogram(self, signal):
        return librosa.stft(signal, n_fft=self.frame_size, hop_length=self.hop_length)

    # Band Energy Ratio
    def _band_energy_ratio(self, spectrogram, sr, split_frequency=2000):
        split_frequency_bin = self._calculate_split_frequency_bin(spectrogram, split_frequency, sr)
        power_spec = self._to_power_spectrogram(spectrogram).T
        return np.array([self._calculate_ber_for_frame(freqs_in_frame, split_frequency_bin) for freqs_in_frame in power_spec])

    def _calculate_split_frequency_bin(self, spectrogram, split_frequency, sr):
        frequency_range = sr / 2
        frequency_delta_per_bin = frequency_range / spectrogram.shape[0]
        split_frequency_bin = np.floor(split_frequency / frequency_delta_per_bin)
        return int(split_frequency_bin)

    def _to_power_spectrogram(self, spectrogram):
        return (np.abs(spectrogram) ** 2)

    def _calculate_ber_for_frame(self, frequencies_in_frame, split_frequency_bin):
        sum_power_low_freq = np.sum(frequencies_in_frame[:split_frequency_bin])
        sum_power_high_freq = np.sum(frequencies_in_frame[split_frequency_bin:])
        return (sum_power_low_freq / sum_power_high_freq)

    # Spectral Centroid
    def _spectral_centroid(self, signal, sr):
        return librosa.feature.spectral_centroid(signal, sr=sr, n_fft=self.frame_size, hop_length=self.hop_length)[0]

    # Spectral Bandwidth
    def _spectral_bandwidth(self, signal, sr):
        return librosa.feature.spectral_bandwidth(signal, sr=sr, n_fft=self.frame_size, hop_length=self.hop_length)[0]

    # Spectral Rolloff
    def _spectral_rolloff(self, signal, sr):
        return librosa.feature.spectral_rolloff(signal, sr=sr, n_fft=self.frame_size, hop_length=self.hop_length)[0]

    # Spectral Flatness
    def _spectral_flatness(self, signal):
        return librosa.feature.spectral_flatness(signal, n_fft=self.frame_size, hop_length=self.hop_length)[0]

    # Spectral Contrast
    def _spectral_contrast(self, signal, sr):
        return librosa.feature.spectral_contrast(signal, sr=sr, n_fft=self.frame_size, hop_length=self.hop_length)[0]

    # Spectral Flux
    def _spectral_flux(self, spectrogram):
        num_frames = spectrogram.shape[1]
        curr_spectra = spectrogram[:, 0]
        curr_spectra_nmlzd = self._normalize(curr_spectra)

        sf = []
        for i in range(1, num_frames):
            frame_sf, curr_spectra_nmlzd = self._compute_spect_flux_for_frame(curr_spectra_nmlzd, spectrogram[:, i])
            sf.append(frame_sf)
            
        return np.array(sf)

    def _normalize(self, spectra):
        return spectra / np.sum(spectra)

    def _compute_spect_flux_for_frame(self, prev_spectra_nmlzd, curr_spectra):
        curr_spectra_nmlzd = self._normalize(curr_spectra)
        frame_sf = np.linalg.norm(curr_spectra_nmlzd - prev_spectra_nmlzd)
        return frame_sf, curr_spectra_nmlzd

### Mel-Frequency Cepstral Coefficients
    def _mfcc(self, signal, sr):
        return librosa.feature.mfcc(signal, n_mfcc=AudioFeatureExtractor.N_MFCC, sr=sr)

### Chroma Vector
    def _chroma(self, signal, sr):
        return librosa.feature.chroma_stft(signal, n_chroma=AudioFeatureExtractor.N_CHROMA, sr=sr, n_fft=self.frame_size, hop_length=self.hop_length)

In [266]:
audio_feature_extractor = AudioFeatureExtractor()

In [267]:
audio_data = audio_feature_extractor.extract(audio_file, genre='Classical')

In [268]:
audio_data.shape

(74,)